# Gosling Demo

In this notebook, I am going to do an example gosling demo. This will include:
1. A basic gosling visualization of gene expression data
2. Several tutorials from the 2022 ISMB demo
3. A heatmap
4. A recreation of [this](https://gosling.js.org/?example=LINKING) visualization using the gos package

In [3]:
# imports
import gosling as gos

#### Warmup: Gene expression data
To start, I am making a basic visual of gene expression data using the BIGWIG ChIP-seq dataset in the gosling documentation. This is to test out different configurations and how they impact a visualization.

Dataset: [BigWig ChIP-seq for H2AZ](https://s3.amazonaws.com/gosling-lang.org/data/4DNFIMPI5A9N.bw), from gosling plain datasets

In [120]:
# create a new track
vis = gos.Track(
    data=gos.BigWigData(
        'bigwig',
        url='https://4dn-open-data-public.s3.amazonaws.com/fourfront-webprod/wfoutput/a57e91de-47f9-4157-9462-50fc78f357e3/4DNFIMPI5A9N.bw',
        # setting this value allows you to update the value of y!
        #column='position'
        value='line'
    ),
).mark_area(
    # mark_area: sets the output to an area graph
    # other option that may apply here --> mark_line
).encode(
    # can use this format for x, or set x to the column setting from the data declaration
    x=gos.X('position:G', domain=gos.DomainChr('chr2')),
    # DomainChr: allows you to zoom on a specific chromosome
    y='line:Q'
).view()

# can also save as html to view in browser!
vis.save('visuals/basic-gene-expr.json')

### Tutorials
In this section, I'm going to learn more about writing in-depth gosling specs via the ISMB tutorials. 

#### Single Track
Data: [BED file](https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.bed) with cytoband information

In [132]:
# import data into csv wrapper
data=gos.csv(
    url='https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.bed',
    separator="\t",
    # identify headers and genomic fields
    headerNames=['chrom', 'chromStart', 'chromEnd', 'name', 'stain'],
    chromosomeField="chrom",
    genomicFields=["chromStart", "chromEnd"]
)

# add data to track and view
view = gos.Track(data).mark_point().encode(
    # apply visual encoding
    x=gos.X('chromStart', type='genomic'),
    # or: x=gos.X('chromStart:G')
    #y=gos.Y('chromStart', type='genomic',axis='left'),
    color=gos.value('lightblue')
).view()

# save as json and html
view.save('visuals/single-track-tutorial.json')
view.save('visuals/single-track-tutorial.html')

Lovely! Now that we have a basic visualization working for gene expression data, we can try adding some complexity in view (ie, colors, legend, etc.)

Next, we are going to try out 


#### Visual Linking!!
The goal of this section is to recreate (this)[https://gosling.js.org/?example=LINKING] visualization using gos

How is this being structured?
- view with 3 tracks
    - one subview (horizontal) and one track

To do this, we can structure from the individual tracks up to the views (recall that alignment happens in the view level!)

We can complete this by writing each track individually, then combining them into a big view. How do you eat an elephant? You chunk it down.

In [115]:
# linear tracks
detail_view=gos.Track(
    data=gos.MultivecData(
        type='multivec',
        url='https://server.gosling-lang.org/api/v1/tileset_info/?d=cistrome-multivec',
        row='sample',
        column='position',
        value='peak',
        # define categories for use in individual overlays!
        categories = ['sample 1', 'sample 2', 'sample 3', 'sample 4']
    ),
    layout='linear',
    linkingId='detail'
).mark_area(
).encode(
    x=gos.X('position:G', axis='top', domain=gos.DomainChr('chr1')),
    y=gos.Y('peak:Q'),
    row=gos.Row('sample:N'),
    color=gos.Color('sample:N')
).view()

detail_view.save('detail_view.html')

In [116]:
# upper left track
circle_brush=gos.Track(
    data=gos.MultivecData(
        type='multivec',
        url='https://server.gosling-lang.org/api/v1/tileset_info/?d=cistrome-multivec',
        row='sample',
        column='position',
        value='peak',
        # define categories for use in individual overlays!
        categories = ['sample 1', 'sample 2', 'sample 3', 'sample 4']
    ),
    static=True,
    layout='circular',
    
).mark_area(
).encode(
    x=gos.X('position:G', domain=gos.DomainChr('chr1')),
    xe=gos.X(field='end', domain=gos.DomainChr('chr1')),
    y='peak:Q',
    row=gos.Row('sample:N'),
    color=gos.Color('sample:N')
).view()


circle_brush.save('upper_left.html')

In [ ]:
# upper right track

          "alignment": "overlay",
          "tracks": [
            {"mark": "bar"},
            {"mark": "brush", "x": {"linkingId": "detail"}}
          ],